In [16]:
!pip install transformers[torch] datasets huggingface_hub



*  Dataset
*  Tokenizer
*  Model

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling

In [3]:
# model_name = "EleutherAI/pythia-70m-deduped" #"EleutherAI/pythia-70m" #gpt2
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model = GPTNeoXForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
from datasets import load_dataset
imdb_dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
imdb_dataset_train = imdb_dataset["train"].select(range(0,500))
imdb_dataset_test = imdb_dataset["test"].select(range(0,500))

In [7]:
def add_content(example):
  label = "Negative" if example["label"] == 0 else "Positive"
  content = example["text"] + "\nSentiment: " + label + tokenizer.eos_token
  return {"content" : content}

In [8]:
train_content = imdb_dataset.map(add_content)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
shuffle_train_content = train_content.shuffle(seed=42)

In [10]:
shuffle_train_content["train"]["label"][0:10]

[1, 1, 0, 1, 0, 1, 1, 0, 0, 1]

In [11]:
print(train_content["train"][0]["content"])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [13]:
def tokenizer_function(example):
  outputs = tokenizer(
        example["content"],
        truncation=True,
        padding=True,
        # max_length=512,
        return_tensors="pt"
    ).to("cuda")

  return outputs

In [14]:
train_tokenized_data = shuffle_train_content.map(tokenizer_function, batched=True)
train_tokenized_data = train_tokenized_data.remove_columns(train_content["train"].column_names)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [44]:
from huggingface_hub import notebook_login, HfApi
notebook_login()
api = HfApi()

In [48]:
api.upload_folder(
    folder_path="ft-imdb-sentiment/checkpoint-100",
    repo_id="santis2/test_distilgpt2_imdb_sentiment",
    create_pr=1,
)

'https://huggingface.co/santis2/test_distilgpt2_imdb_sentiment/tree/refs%2Fpr%2F1/'

In [18]:
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)

In [42]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="ft-imdb-sentiment",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    eval_steps=1_00,
    logging_steps=1_00,
#    gradient_accumulation_steps=8,
    num_train_epochs=2,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1_00,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=train_tokenized_data["train"].select(range(0,1000)),
    eval_dataset=train_tokenized_data["test"].select(range(0,100))
)


RepositoryNotFoundError: ignored

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub("test_distilgpt2_imdb_sentiment")

In [31]:
from transformers import pipeline

In [32]:
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, device="cuda"
)

In [35]:
index = 24000
inputs = train_content["test"][index]["text"]+"\nSentiment:"
print(pipe(inputs, do_sample=True,temperature=0.6,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id)[0]["generated_text"])
print(train_content["test"][index]["label"])

The movie is about a girl who's not going to a bonfire only because she's baby-sitting that night. Nothing weird about that, right? Until ... The phone rings. Until ... The phone rings again. And again ... And again. Those are not some stupid prank calls. This is for real. If you wanna see how the girl reacts, just watch the movie.<br /><br />Great atmosphere filled with scary sounds. Very well performed by young Camilla Belle who got the lead role. I see in her some great potential to become a good actress. This is more than only a decent thriller, I have no idea why it's so underrated. Anyway, on my opinion this movie deserves more than only 4/10. 24% of all voters rated the movie with 1. Get serious, people. You couldn't get a better thriller for a title like this.
Sentiment: Positive
1


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1262: UserWarning: Input length of input_ids is 184, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [49]:
from transformers import pipeline

In [50]:
pipe = pipeline("text-generation",model="santis2/test_distilgpt2_imdb_sentiment")

In [52]:
index = 2400
inputs = train_content["test"][index]["text"]+"\nSentiment:"
print(pipe(inputs, do_sample=True,temperature=0.6,eos_token_id=tokenizer.eos_token_id,pad_token_id=tokenizer.eos_token_id)[0]["generated_text"])
print(train_content["test"][index]["label"])

seriously, if i wanted to make a movie that makes zero sense, never will, and features lesbian scenes as its only high-point, i could have.<br /><br />david lynch is the worst, as is this movie. anyone could have made a better movie in which at least some answers were given and the story wasn't so slow and long-winded. the story means nothing without something at the end besides the credits. what a waste of time. i will never get those 147 minutes of my life back and hope that others can learn from my mistake.
Sentiment: Negative
0


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1262: UserWarning: Input length of input_ids is 123, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
